In [ ]:
!pip install langchain-community

In [5]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.embeddings import OpenAIEmbeddings # Importing OpenAI embeddings from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain.vectorstores.chroma import Chroma # Importing Chroma vector store from Langchain
from dotenv import load_dotenv # Importing dotenv to get API key from .env file
from langchain.chat_models import ChatOpenAI # Import OpenAI LLM
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations
from langchain_community.document_loaders import TextLoader

# Path to the directory to save Chroma database
CHROMA_PATH = "chroma"

In [13]:
# Directory to your pdf files:
DATA_PATH = "/articles/"
def load_documents():
  """
  Load PDF documents from the specified directory using PyPDFDirectoryLoader.
  Returns:
  List of Document objects: Loaded PDF documents represented as Langchain
                                                          Document objects.
  """
  # Initialize PDF loader with specified directory
  loader = TextLoader("articles/doc_craft.txt")

  return loader.load() 

documents = load_documents() # Call the function
# Inspect the contents of the first document as well as metadata
print(documents)

[Document(metadata={'source': 'articles/doc_craft.txt'}, page_content='ntroduction\nYou are a Data Scientist struggling with data, code, and models in your projects.\n\nYou are an ML Engineer who has trouble replicating pipelines and monitoring models in production.\n\nYou are Head of a Data Science team, who has trouble shipping quickly models to production.\n\nYou are a CTO who has difficulty to move Python code into large-scale production and manage the DevOps workload on an AI project.\n\n🚀 Then, you are a potential user of Craft AI\'s MLOps platform, which aims to accelerate the deployment and improve the management of your Machine Learning models in production.\n\nFind all the information you need\n Get started\n\nLearn the basics and launch your first script on the platform.\n\n Getting started\n\n User Workflow\n\nList of user workflows for creating deployments, using data, customising your code, etc.\n\n Introduction user worklflow\n\n Step & Pipeline\n\nUnderstand the ML proc

In [14]:
def split_text(documents: list[Document]):
  """
  Split the text content of the given list of Document objects into smaller chunks.
  Args:
    documents (list[Document]): List of Document objects containing text content to split.
  Returns:
    list[Document]: List of Document objects representing the split text chunks.
  """
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

In [15]:

def save_to_chroma(chunks: list[Document]):
  """
  Save the given list of Document objects to a Chroma database.
  Args:
  chunks (list[Document]): List of Document objects representing text chunks to save.
  Returns:
  None
  """

  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_documents(
    chunks,
    OpenAIEmbeddings(),
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [16]:
def generate_data_store():
  """
  Function to generate vector database in chroma from documents.
  """
  documents = load_documents() # Load documents from a source
  chunks = split_text(documents) # Split documents into manageable chunks
  save_to_chroma(chunks) # Save the processed data to a data store

# Load environment variables from a .env file
load_dotenv()
# Generate the data store
generate_data_store()

Split 1 documents into 1251 chunks.
ntroduction
You are a Data Scientist struggling with data, code, and models in your projects.

You are an ML Engineer who has trouble replicating pipelines and monitoring models in production.

You are Head of a Data Science team, who has trouble shipping quickly models to production.
{'source': 'articles/doc_craft.txt', 'start_index': 0}


/home/hugo/miniconda3/envs/scw_rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


Saved 1251 chunks to chroma.


/home/hugo/miniconda3/envs/scw_rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [2]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
 - -
Answer the question based on the above context: {question}
"""

In [6]:
from langchain_core.prompts import ChatPromptTemplate

def query_rag(query_text):
  """
  Query a Retrieval-Augmented Generation (RAG) system using Chroma database and OpenAI.
  Args:
    - query_text (str): The text to query the RAG system with.
  Returns:
    - formatted_response (str): Formatted response including the generated text and sources.
    - response_text (str): The generated response text.
  """
  # YOU MUST - Use same embedding function as before
  embedding_function = OpenAIEmbeddings()

  # Prepare the database
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)
  
  # Retrieving the context from the DB using similarity search
  results = db.similarity_search_with_relevance_scores(query_text, k=10)

  # Check if there are any matching results or if the relevance score is too low
  if len(results) == 0 or results[0][1] < 0.7:
    print(f"Unable to find matching results.")

  # Combine context from matching documents
  context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])
 
  # Create prompt template using context and query text
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query_text)
  
  # Initialize OpenAI chat model
  model = ChatOpenAI(model="gpt-4o-mini")

  # Generate response text based on the prompt
  response_text = model.predict(prompt)
 
   # Get sources of the matching documents
  sources = [doc.metadata.get("source", None) for doc, _score in results]
 
  # Format and return response including generated text and sources
  formatted_response = f"Response: {response_text}\nSources: {sources}"
  return formatted_response, response_text



In [7]:
query_text = "How to use create_pipeline ? "

# Let's call our function we have defined
formatted_response, response_text = query_rag(query_text)
# and finally, inspect our final response!
print(response_text)
# formatted_response

/home/hugo/miniconda3/envs/scw_rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(
/home/hugo/miniconda3/envs/scw_rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/hugo/miniconda3/envs/scw_rag/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecatio

To use the `create_pipeline` method, you need to call it with the required parameters: `pipeline_name` and `step_name`. Here’s an example of how to do it:

```python
sdk.create_pipeline(
    pipeline_name="your_pipeline_name",
    step_name="your_step_name"
)
```

Replace `"your_pipeline_name"` with the desired name for your pipeline and `"your_step_name"` with the name for the step in the pipeline. After executing this function, you will obtain an output that describes the pipeline, its step, and its inputs and outputs.


# For testing fastaapi server

In [7]:
from openai import OpenAI

# init client and connect to localhost server
client = OpenAI(
    api_key="fake-api-key",
    base_url="http://127.0.0.1:8001" # change the default port if needed
)

# call API
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "How many $ have Hugo on him bank account ?",
        }
    ],
    model="gpt-1337-turbo-pro-max",
)

# print the top "choice" 
print(chat_completion.choices[0].message.content)

Hugo has 10$ on his bank account.


In [2]:


from openai import OpenAI

# init client and connect to localhost server
client = OpenAI(
    api_key="fake-api-key",
    base_url="http://127.0.0.1:8000" # change the default port if needed
)

# call API
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            # "content": "How to create a pipeline ?",
            "content": "How many $ have Hugo on him bank account ?",
        }
    ],
    model="gpt-1337-turbo-pro-max",
)

# print the top "choice" 
print(chat_completion.choices[0].message.content)

Hugo has $10 in his bank account.


In [11]:
from openai import OpenAI

client = OpenAI(
    api_key="fake-api-key",
    base_url="http://localhost:8000" # change the default port if needed
)


stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say this is a test, s greater than number of elements in index 6, updating n_results = 6"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

The context provided does not contain any information relevant to the message you shared. Therefore, I cannot answer the question based on the given context. 